# Creating a logistic regression to predict absenteeism


In [1]:
import numpy as np
import pandas as pd

In [2]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [3]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


## Create the targets


In [4]:
#Anything below the median will be 0
#Anything above the median will be 1

data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [5]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [6]:
data_preprocessed['Excessive Absenteeism'] = targets

In [7]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [8]:
targets.sum()/targets.shape[0]

0.45571428571428574

In [9]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'], axis=1)

In [10]:
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,0


## Select inputs for the regression


In [11]:
data_with_targets.shape

(700, 15)

In [12]:
unscaled_inputs = data_with_targets.iloc[:,:-1]
unscaled_inputs.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1


## Standardize the date


In [13]:
#Scale only the numerical features

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

In [14]:
class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [15]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Day of the week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [16]:
columns_to_scale = ['Month Value',
       'Day of the week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets']

In [17]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [18]:
absenteeism_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month Value', 'Day of the week',
                      'Transportation Expense', 'Distance to Work', 'Age',
                      'Daily Work Load Average', 'Body Mass Index', 'Education',
                      'Children', 'Pets'],
             copy=None, with_mean=None, with_std=None)

In [19]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [20]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,-0.683704,1.005844,0.412816,-0.536062,-0.806331,0.767431,-0.447980,0.880469,0.268487
1,0,0,0,0,0.182726,-0.683704,-1.574681,-1.141882,2.130803,-0.806331,1.002633,-0.447980,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.007725,-0.654143,1.426749,0.248310,-0.806331,1.002633,-0.447980,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.668253,0.854936,-1.682647,0.405184,-0.806331,-0.643782,-0.447980,0.880469,-0.589690
4,0,0,0,1,0.182726,0.668253,1.005844,0.412816,-0.536062,-0.806331,0.767431,-0.447980,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.007725,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,2.232242,0.880469,-0.589690
696,1,0,0,0,-0.388293,-0.007725,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,-0.447980,-0.019280,1.126663
697,1,0,0,0,-0.388293,0.668253,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,2.232242,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.668253,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,2.232242,-0.919030,-0.589690


## Split the data into train and test & and shuffle

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
train_test_split(scaled_inputs, targets)

[     Reason_1  Reason_2  Reason_3  Reason_4  Month Value  Day of the week  \
 495         0         0         0         1     0.753746        -0.683704   
 319         1         0         0         0     1.324766        -0.683704   
 659         0         0         0         1    -0.673803         1.344231   
 601         0         0         0         1    -1.244823         1.344231   
 90          0         0         0         1     1.324766        -0.007725   
 ..        ...       ...       ...       ...          ...              ...   
 226         0         0         0         1    -0.102784        -1.359682   
 109         0         0         0         1     1.610276        -0.007725   
 363         0         0         0         1    -1.530333        -1.359682   
 431         0         0         0         1    -0.388293         1.344231   
 345         1         0         0         0     1.610276         0.668253   
 
      Transportation Expense  Distance to Work       Age  \
 4

In [23]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, test_size = 0.2, random_state=42)

## Logistic regression


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [25]:
reg = LogisticRegression(verbose=2)

In [26]:
reg.fit(x_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


LogisticRegression(verbose=2)

In [27]:
reg.score(x_train, y_train)

0.7660714285714286

## Checking accuracy

In [28]:
model_outputs = reg.predict(x_train)

In [29]:
model_outputs == y_train

array([ True,  True,  True,  True, False, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True, False,  True, False,  True,
        True, False, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True, False,
        True,  True, False, False,  True,  True, False, False,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True, False,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [30]:
np.sum((model_outputs == y_train))/model_outputs.shape[0]*100

76.60714285714286

In [31]:
reg.intercept_

array([-1.7575216])

In [32]:
reg.coef_

array([[ 2.92993196,  0.73453473,  3.07517417,  0.99389691,  0.07923215,
        -0.15754023,  0.67570676, -0.05915557, -0.25914578, -0.02183528,
         0.24334102, -0.10857391,  0.41638209, -0.31223952]])

In [33]:
feature_name = unscaled_inputs.columns.values

In [34]:
summary_table = pd.DataFrame(columns=['Feature Name'], data=feature_name)
summary_table["Coefficient"] = np.transpose(reg.coef_)
summary_table

,Feature Name,Coefficient
0,Reason_1,2.929932
1,Reason_2,0.734535
2,Reason_3,3.075174
3,Reason_4,0.993897
4,Month Value,0.079232
5,Day of the week,-0.157540
6,Transportation Expense,0.675707
7,Distance to Work,-0.059156
8,Age,-0.259146
9,Daily Work Load Average,-0.021835


In [35]:
summary_table.index = summary_table.index + 1

In [36]:
summary_table.loc[0] = ["Intercept", reg.intercept_[0]]

In [37]:
summary_table.sort_values('Coefficient', ascending=False)

,Feature Name,Coefficient
3,Reason_3,3.075174
1,Reason_1,2.929932
4,Reason_4,0.993897
2,Reason_2,0.734535
7,Transportation Expense,0.675707
13,Children,0.416382
11,Body Mass Index,0.243341
5,Month Value,0.079232
10,Daily Work Load Average,-0.021835
8,Distance to Work,-0.059156


## Testing the model


In [38]:
reg.score(x_test, y_test)

0.7714285714285715

In [40]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.82479129, 0.17520871],
       [0.87647778, 0.12352222],
       [0.77483419, 0.22516581],
       [0.58309947, 0.41690053],
       [0.51095095, 0.48904905],
       [0.08342333, 0.91657667],
       [0.66267711, 0.33732289],
       [0.3461606 , 0.6538394 ],
       [0.75267385, 0.24732615],
       [0.71080784, 0.28919216],
       [0.8873699 , 0.1126301 ],
       [0.68878666, 0.31121334],
       [0.25475123, 0.74524877],
       [0.51186407, 0.48813593],
       [0.74009104, 0.25990896],
       [0.45183015, 0.54816985],
       [0.92160616, 0.07839384],
       [0.25305153, 0.74694847],
       [0.88831271, 0.11168729],
       [0.59781606, 0.40218394],
       [0.67248635, 0.32751365],
       [0.78027914, 0.21972086],
       [0.67318629, 0.32681371],
       [0.66823202, 0.33176798],
       [0.87004581, 0.12995419],
       [0.18751338, 0.81248662],
       [0.60454451, 0.39545549],
       [0.56493446, 0.43506554],
       [0.79462288, 0.20537712],
       [0.62889024, 0.37110976],
       [0.

## Saving the model

In [41]:
import pickle

In [42]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)
 #'rm' for reading
 #pickle.load for loading the model    

In [43]:
with open('scaler', 'wb') as file:
    pickle.dump(absenteeism_scaler, file)